In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Preparing the environment

In [19]:
import os
import sys
import torch

!curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
!tar xzf 1.10.0.tar.gz
os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
!pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

In [20]:
import pytorch3d as p3d 
print(p3d.__version__)

In [21]:
import torch
import matplotlib.pyplot as plt

# Util function for loading meshes
import pytorch3d.structures as py3d_struct
from pytorch3d.io import load_objs_as_meshes, load_obj

from pytorch3d.ops import sample_points_from_meshes

# Data structures and functions for rendering
from pytorch3d.structures import Meshes
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.vis.texture_vis import texturesuv_image_matplotlib
from pytorch3d.renderer import (look_at_view_transform,
                                OpenGLPerspectiveCameras, 
                                FoVPerspectiveCameras, 
                                PointLights, 
                                DirectionalLights, 
                                Materials, 
                                RasterizationSettings, 
                                MeshRenderer, 
                                MeshRasterizer,  
                                SoftPhongShader,
                                TexturesUV,
                                TexturesVertex,
                                HardPhongShader)

# add path for demo utils functions 
import sys
import os
sys.path.append(os.path.abspath(''))

In [22]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

from IPython.display import HTML

If you have got a CUDA device, you can use GPU mode

In [23]:
if torch.cuda.is_available():
  device = torch.device('cuda:0')
  torch.cuda.set_device(device)
else:
  device = torch.device('cpu')

Downloading the 3D Models

In [24]:
!wget -P Models3D/Geralt https://raw.githubusercontent.com/Aynur19/3D-ML/main/Models3D/Geralt/Geralt.obj
!wget -P Models3D/Geralt https://raw.githubusercontent.com/Aynur19/3D-ML/main/Models3D/Geralt/Geralt.mtl

# Mesh Rendering

Loading and preparing a 3D model for rendering

In [25]:
mesh = load_objs_as_meshes(['./Models3D/Geralt/Geralt.obj'], device=device)

verts = mesh.verts_packed()
N = verts.shape[0]
center = verts.mean(0)
scale = max((verts - center).abs().max(0)[0])
mesh.offset_verts_(-center)
mesh.scale_verts_((1.0 / float(scale)));

verts_rgb = torch.ones_like(verts)[None]  # (1, V, 3)
faces = mesh.faces_packed()
textures = TexturesVertex(verts_features=verts_rgb.to(device))

mesh = Meshes(
    verts=[verts.to(device)],   
    faces=[faces.to(device)], 
    textures=textures
)

In [26]:
def getMeshRenders(mesh, distance=2.5, elevation=0.0, maxAzimuth=360, rangeAzimuth=2):
  allImages = []
  for azimuth in range(0, maxAzimuth, rangeAzimuth):

    R, T = look_at_view_transform(distance, elevation, azimuth) 
    cameras = FoVPerspectiveCameras(device=device, R=R, T=T)

    raster_settings = RasterizationSettings(
        image_size=1024, 
        blur_radius=0.0, 
        faces_per_pixel=1, 
    )

    lights = DirectionalLights(device=device)

    renderer = MeshRenderer(rasterizer=MeshRasterizer(cameras=cameras, raster_settings=raster_settings),
                            shader=SoftPhongShader(device=device, cameras=cameras, lights=lights)
    )

    allImages.append(renderer(mesh)[0, ..., :3].cpu().numpy())

  return allImages;

In [27]:
def getMeshRenderingImages(images):
    fig, ax = plt.subplots()
    fig.set_figheight(10)
    fig.set_figwidth(10)
    ax.axis('off')
    ims = []
  
    for i in range(len(images)):
        im = ax.imshow(images[i], animated=True)
        if i == 0:
            ax.imshow(images[i])
        ims.append([im])
    plt.close()

    return fig, ims

Getting render images from different angles

In [28]:
images = getMeshRenders(mesh)
fig, ims = getMeshRenderingImages(images)

Getting animation and displaying

In [29]:
anim = animation.ArtistAnimation(fig, ims, interval=25, blit=True, repeat_delay=2000)
HTML(anim.to_html5_video())

Saving animation

In [30]:
anim.save('Geralt. Mesh Rendering Result.mp4', fps=30, extra_args=['-vcodec', 'libx264'])

# Point Cloud Rendering

In [35]:
points = sample_points_from_meshes(mesh, 10000)
points = points.clone().detach().cpu().squeeze().unbind(1)

In [36]:
def plotPointCloud():
    ax.scatter(x, z, -y, marker='.')  
    return fig,
    
def animatePlot3D(i):
    ax.view_init(elev=180., azim=i)
    return fig,

Getting animation and displaying

In [38]:
fig = plt.figure(figsize=(10, 10))
ax = Axes3D(fig)
ax.axis('off')
x, y, z = points

anim = animation.FuncAnimation(fig, animatePlot3D, init_func=plotPointCloud, frames=360, interval=20, blit=True)
plt.close()
HTML(anim.to_html5_video())

Saving animation

In [39]:
anim.save('Geralt. Point Cloud Rendering Result.mp4', fps=30, extra_args=['-vcodec', 'libx264'])